In [1]:
import numpy as np
import os
import cv2
from tqdm import tqdm
from multiprocessing import Pool
# jupyter无法使用多进程！

In [2]:
input_path = "../../data/DIV2K_HR/DIV2K_train_LR_bicubic/X4"

In [3]:
img_name = os.listdir(input_path)

In [4]:
imgs_path_list = []
for i, name in enumerate(img_name):
    str1 = os.path.join(input_path, name)
    imgs_path_list.append(str1)

In [15]:
def img_print(img_path):
    with open("./test.txt", "w") as f:
        f.write(img_path)
    process_info = f'Processing {img_path} ...'
    return process_info

In [16]:
pbar = tqdm(total=len(imgs_path_list), unit="image", desc="extract")

extract:   0%|          | 0/800 [05:30<?, ?image/s]


In [ ]:
pbar.update(1)

In [17]:
pool = Pool(20)

In [ ]:
for path in imgs_path_list:
    pool.apply_async(img_print, args=path, callback=lambda arg:pbar.update(1))
pool.close()
pool.join()
pbar.close()

In [6]:
name = os.path.basename(imgs_path_list[0])

In [7]:
os.path.splitext(name)

('0001x4', '.png')

In [9]:
img_name, suffix = name.split(".")

In [10]:
img_name = img_name.replace("x2", "").replace("x3", "").replace("x4", "")

In [14]:
# -1表示读入原图像，即若是彩色图像读入便为彩色图像。若为灰度图读入便为灰度图
img = cv2.imread(imgs_path_list[0], -1)
h, w, c = img.shape

In [15]:
crop_size = 120
step = 60

In [19]:
h_sub_point = np.arange(0, h - crop_size + 1, step)
w_sub_point = np.arange(0, w- crop_size + 1, step)

In [20]:
# 若最后分割起始点小于h-crop_size的长度，则加入位于h-crop_size处的分割起始点
if h - (h_sub_point[-1] + crop_size) > 0:
    h_sub_point = np.append(h_sub_point, h - crop_size)
if w - (w_sub_point[-1] + crop_size) > 0:
    w_sub_point = np.append(w_sub_point, w - crop_size)


In [22]:
index = 0
for x in h_sub_point:
    for y in w_sub_point:
        index += 1
        crop_img = img[x:x+crop_size, y:y+crop_size, :]
        crop_img = np.ascontiguousarray(crop_img)
        cv2.imwrite(os.path.join("./test_img", "{}_{}.{}".format(img_name, index, suffix)), crop_img
                    , [cv2.IMWRITE_PNG_COMPRESSION, 3])